# Neural Network
Code based on Tariq Rashid Make your own Neural Network Book

In [99]:
import scipy.special
import numpy
import matplotlib.pyplot
%matplotlib inline

In [100]:
# Neural Network Class
class neuralNetwork:
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # set number of nodes in each input, hidden, output layer
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes        
        
        # learning rate
        self.lr = learningrate
        
        # using a sigmoid activation function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        # set initial weights using a normal distribution with st.dev = 1/sqrt(income_links)
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        self.who = numpy.random.normal(0.0, pow(self.hnodes, -0.5), (self.onodes, self.hnodes))

    # train the neural network
    def train(self, inputs_list, targets_list):
        # I) Start by passing the values trought the network
        # 1) convert inputs list to 2d array and transpose
        inputs = numpy.array(inputs_list, ndmin=2).T
        targets = numpy.array(targets_list, ndmin=2).T
        
        # 2) calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 3) calculate the signals emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 4) calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 5) calculate the signals emerging from final output layer
        final_outputs = self.activation_function(final_inputs)
        
        # II) Get the errors and run it again
        # 1) output layer error is the (target - actual)
        output_errors = targets - final_outputs
        # 2) hidden layer error is the output_errors split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors) 
        
        # 3) output the weights for the links between the hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        # 4) update the weights for the links between the input and the hidden layres
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
    
    # query the neural network
    def query(self, inputs_list):
        # 1) convert inputs list to 2d array and transpose
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        # 2) calculate signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        # 3) calculate signlas emerging from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        
        # 4) calculate signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        # 5) calculate signals emerging from the final output
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs 

In [101]:
# Train the NN
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

# learning rate
learning_rate = 0.1

# create instance of neural network
n = neuralNetwork(input_nodes,hidden_nodes,output_nodes, learning_rate)

# load the mnist training data CSV file into a list
training_data_file = open('data/mnist_train_100.csv', 'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

# train it!
# epochs is the number of times the training data set is used for training
epochs = 5

for e in range(epochs):
    # go through all records in the training data set
    for record in training_data_list:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:])/255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
    

In [102]:
# Test the NN
# load the mnist test data CSV file into a list
test_data_file = open("data/mnist_test_10.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_data_list:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

# calculate the performance score
scorecard_array = numpy.asarray(scorecard)
print("performace = ", scorecard_array.sum()/scorecard_array.size)


# draw the number
#image_array = numpy.asfarray(all_values[1:]).reshape((28,28))
#matplotlib.pyplot.imshow(image_array, cmap='Greys', interpolation='None')

performace =  0.6
